# Vertices for Hubbard atom

In [ ]:
using Revise
using LinearAlgebra
using SparseIR
import SparseIR: valueim
using MSSTA
import MSSTA: QuanticsInd, quantics_to_index, asqubits, index_to_quantics, QubitInd, qubit_to_index, index_to_qubit
using OvercompleteIR
import OvercompleteIR: PHConvention, freq_box
import OvercompleteIR.Atom: HubbardAtom, MagneticChannel, chi0, full_vertex, gamma
using Plots
import TensorCrossInterpolation as TCI

newaxis = [CartesianIndex()]

In [ ]:
_delta(i, j) = (i == j ? 1 : 0)

In [ ]:
using ITensors

In [ ]:
U = 5.0
beta  = 1.0
ch = DensityChannel()
conv = PHConvention()
model = HubbardAtom(U, beta)

Nf = 2^9
Nb = 10
wb = BosonicFreq(2*Nb)
box = freq_box(conv, 2*Nf, 2*Nb+1)
box = reshape(box, (2*Nf, 2*Nf, 2*Nb+1))
;

In [ ]:
data = gamma.(ch, model, box[:, :, end])
;

In [ ]:
heatmap(abs.(data))

In [ ]:
# TCI with R bits
function create_func(func, R, wforg, wb)
    function q_to_n(q::Vector{QubitInd})
        tmp = BosonicFreq.(2 .* (qubit_to_index(Val(2), q, R) .- 1)) .+ wforg
        return (tmp..., wb)
    end
    function func_q(q::Vector{QubitInd})
        return func(q_to_n(q))
    end
    return q_to_n, func_q
end

R = 10
q2n, fq = create_func(w->gamma(ch, model, w), R, FermionicFreq(-2Nf+1), wb)

In [ ]:
fI = x->fq(QubitInd.(x))
localdims = fill(2, 2*R)

firstpivot = TCI.optfirstpivot(fI, localdims)
@show fI(firstpivot)

In [ ]:
qtt, ranks, errors = TCI.crossinterpolate2(
    ComplexF64, fI, localdims,
    [firstpivot], tolerance=1e-6, maxiter=10000, verbosity=1
)

In [ ]:
q_diagonal = [index_to_qubit((i, 2^(R-1)), R) for i in 1:2^(R-6):2^(R-1)]
reconst_diagonal = [TCI.evaluate(qtt, convert.(Int, q)) for q in q_diagonal]
ref_diagonal = fq.(q_diagonal)
;

In [ ]:
p = plot(yaxis=:log, ylims=(1e-10,1e+2))
plot!(p, abs.(reconst_diagonal), marker=:x)
plot!(p, abs.(ref_diagonal .- reconst_diagonal))